In [ ]:
# Standard Libraries
import os
import time
import gc
import joblib  # For saving models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.feature import hog

# Parallel Processing
from joblib import Parallel, delayed

# Machine Learning
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Metrics & Evaluation
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


In [62]:
# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully from NumPy files.")

Loading dataset...
Training Data loaded successfully from NumPy files.


**efficiently preprocesses images** by normalizing, resizing, and converting them into a format that machine learning models can handle **with minimal memory usage**.

In [68]:
# Preprocessing function (optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float32') / 255.0  # Normalize
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float32)
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

In this research we have explored the utlisation of five machine learning models:
 
- **Stochastic Gradient Descent (SGD) Classifier**: A linear model optimized using gradient descent, well-suited for large-scale datasets.  
- **Support Vector Machine (SVM)**: A powerful classifier that finds the optimal decision boundary to separate different classes.  
- **Random Forest Classifier**: An ensemble learning method that builds multiple decision trees to improve accuracy and reduce overfitting.  
- **XGBoost Classifier**: A highly efficient gradient boosting algorithm that enhances predictive performance through boosting techniques.  
- **Gradient Boosting Classifier**: A sequential ensemble learning method that builds trees iteratively to minimize errors and improve prediction accuracy.  

In [72]:
# Apply preprocessing with minimal memory usage
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder if it doesn't exist
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)
print("Models directory set up.")


# Define lightweight models
models = {
    "SGD": Pipeline([("scaler", StandardScaler()), ("sgd", SGDClassifier(loss="log_loss", max_iter=50, tol=1e-3))]),
    "SVM": Pipeline([("scaler", StandardScaler()), ("svm", SVC(kernel="linear", probability=False, cache_size=50))]),
    "RandomForest": RandomForestClassifier(n_estimators=30, max_depth=4, min_samples_split=5, n_jobs=-1),
    "XGBoost": XGBClassifier(n_estimators=20, max_depth=3, learning_rate=0.1, verbosity=0, use_label_encoder=False)
}



Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Models directory set up.


In [73]:
# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    
    print(f"{name} model training completed in {train_time:.2f}s.")
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)

    # Save model to "models" directory
    model_path = os.path.join(models_dir, f"{name}1.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # Convert to KB

    results[name] = {"Accuracy": acc, "Train Time": train_time, "Model Size (KB)": model_size}
    
    print(f"{name}: Accuracy={acc:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

Training SGD model...


/shared/venv/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD model training completed in 27.55s.
SGD: Accuracy=0.4384, Train Time=27.55s, Model Size=224.35 KB
Training SVM model...
SVM model training completed in 1291.84s.
SVM: Accuracy=0.4387, Train Time=1291.84s, Model Size=326480.53 KB
Training RandomForest model...
RandomForest model training completed in 3.99s.
RandomForest: Accuracy=0.5397, Train Time=3.99s, Model Size=87.19 KB
Training XGBoost model...
XGBoost model training completed in 35.34s.
XGBoost: Accuracy=0.7173, Train Time=35.34s, Model Size=115.28 KB


In [74]:
# Free memory
print("Freeing memory...")
del train_images, train_labels, val_images, val_labels, model
gc.collect()  # Manually trigger garbage collection
print("Memory freed.")

# Output best model
best_model = max(results, key=lambda x: results[x]["Accuracy"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f}")

Freeing memory...
Memory freed.
Best model: XGBoost with Accuracy: 0.7173


In [47]:
# Preprocessing function (optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float32') / 255.0  # Normalize
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float32)
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

In [48]:
# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully from NumPy files.")

Loading dataset...
Training Data loaded successfully from NumPy files.


In [49]:
# Apply preprocessing with minimal memory usage
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.


In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Define optimized models
models = {
    "SGD": Pipeline([("scaler", MinMaxScaler()), ("sgd", SGDClassifier(loss="log_loss", max_iter=11000, tol=1e-3))]),
    "SVM": Pipeline([("scaler", MinMaxScaler()), ("svm", SVC(kernel="linear", probability=False, cache_size=50, max_iter=5000))]),
    "RandomForest": RandomForestClassifier(n_estimators=10, max_depth=3, min_samples_split=5, n_jobs=-1),
    # Optimized XGBoost model with improved hyperparameters
    "XGBoost": XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.08, colsample_bytree=0.8, subsample=0.9, verbosity=0, use_label_encoder=False)

}


In [51]:
# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    
    print(f"{name} model training completed in {train_time:.2f}s.")
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)

    # Save model to "models" directory
    model_path = os.path.join(models_dir, f"{name}2.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # Convert to KB

    results[name] = {"Accuracy": acc, "Train Time": train_time, "Model Size (KB)": model_size}
    
    print(f"{name}: Accuracy={acc:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

Training SGD model...
SGD model training completed in 20.01s.
SGD: Accuracy=0.3707, Train Time=20.01s, Model Size=204.21 KB
Training SVM model...


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM model training completed in 421.04s.
SVM: Accuracy=0.3812, Train Time=421.04s, Model Size=341956.81 KB
Training RandomForest model...
RandomForest model training completed in 0.28s.
RandomForest: Accuracy=0.5397, Train Time=0.28s, Model Size=20.41 KB
Training XGBoost model...
XGBoost model training completed in 72.21s.
XGBoost: Accuracy=0.8665, Train Time=72.21s, Model Size=370.22 KB


In [52]:
# Free memory
print("Freeing memory...")
del train_images, train_labels, val_images, val_labels, model
gc.collect()  # Manually trigger garbage collection
print("Memory freed.")

# Output best model
best_model = max(results, key=lambda x: results[x]["Accuracy"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f}")

Freeing memory...
Memory freed.
Best model: XGBoost with Accuracy: 0.8665


In [53]:


# Preprocessing function (optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float32') / 255.0  # Normalize
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float32)
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully from NumPy files.")

# Apply preprocessing with minimal memory usage
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder if it doesn't exist
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)
print("Models directory set up.")

# Define optimized models
models = {
    "SGD": Pipeline([("scaler", StandardScaler()), ("sgd", SGDClassifier(loss="log_loss", max_iter=10000, tol=1e-3))]),
    "SVM": Pipeline([("scaler", MinMaxScaler()), ("svm", SVC(kernel="linear", probability=False, cache_size=50, max_iter=5000))]),
    "RandomForest": RandomForestClassifier(n_estimators=10, max_depth=3, min_samples_split=5, n_jobs=-1),
    # Further optimized XGBoost model with adjusted hyperparameters
    "XGBoost": XGBClassifier(n_estimators=75, max_depth=5, learning_rate=0.07, colsample_bytree=0.85, subsample=0.95, verbosity=0, use_label_encoder=False)

}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    
    print(f"{name} model training completed in {train_time:.2f}s.")
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)

    # Save model to "models" directory
    model_path = os.path.join(models_dir, f"{name}3.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # Convert to KB

    results[name] = {"Accuracy": acc, "Train Time": train_time, "Model Size (KB)": model_size}
    
    print(f"{name}: Accuracy={acc:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

# Free memory
print("Freeing memory...")
del train_images, train_labels, val_images, val_labels, model
gc.collect()  # Manually trigger garbage collection
print("Memory freed.")

# Output best model
best_model = max(results, key=lambda x: results[x]["Accuracy"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f}")


Loading dataset...
Training Data loaded successfully from NumPy files.
Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Models directory set up.
Training SGD model...
SGD model training completed in 292.89s.
SGD: Accuracy=0.3892, Train Time=292.89s, Model Size=224.35 KB
Training SVM model...


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM model training completed in 381.53s.
SVM: Accuracy=0.3812, Train Time=381.53s, Model Size=341956.81 KB
Training RandomForest model...
RandomForest model training completed in 0.21s.
RandomForest: Accuracy=0.5440, Train Time=0.21s, Model Size=20.00 KB
Training XGBoost model...
XGBoost model training completed in 52.03s.
XGBoost: Accuracy=0.8940, Train Time=52.03s, Model Size=740.09 KB
Freeing memory...
Memory freed.
Best model: XGBoost with Accuracy: 0.8940


### Model Performance Comparison  

| Model         | Run | Accuracy | Train Time (s) | Model Size (KB) |
|--------------|-----|----------|---------------|-----------------|
| **SGD**      | 1   | 0.4384   | 27.55         | 224.35          |
| **SVM**      | 1   | 0.4387   | 1291.84       | 326480.53       |
| **RandomForest** | 1   | 0.5397   | 3.99          | 87.19           |
| **XGBoost**  | 1   | 0.7173   | 35.34         | 115.28          |
| **SVM**      | 2   | 0.3812   | 421.04        | 341956.81       |
| **RandomForest** | 2   | 0.5397   | 0.28          | 20.41           |
| **XGBoost**  | 2   | 0.8665   | 72.21         | 370.22          |
| **SVM**      | 3   | 0.3812   | 381.53        | 341956.81       |
| **RandomForest** | 3   | 0.5440   | 0.21          | 20.00           |
| **XGBoost**  | 3   | 0.8940   | 52.03         | 740.09          |

### Best Model  

**XGBoost** emerges as the best model due to:  
1. **Highest Accuracy**: 0.8940, outperforming other models.  
2. **Reasonable Training Time**: Much faster than SVM while achieving better accuracy.  
3. **Manageable Model Size**: Smaller than SVM, making it more efficient for deployment.  

Given these factors, **XGBoost provides the best trade-off between accuracy, training efficiency, and storage size.**

### More model Optimization, adjusting the hyperparameter values

In [27]:
# Preprocessing function (further optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float32') / 255.0  # Normalize
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float32)
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully from NumPy files.")

# Apply optimized preprocessing
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder if it doesn't exist
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)
print("Models directory set up.")

# Define optimized models
models = {
    "SGD": Pipeline([
        ("scaler", MinMaxScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0001, penalty="l2", max_iter=5000, tol=1e-4))
    ]),
    "SVM": Pipeline([
        ("scaler", MinMaxScaler()), 
        ("svm", SVC(kernel="linear", probability=False, cache_size=50, max_iter=3000))
    ]),
    "RandomForest": RandomForestClassifier(n_estimators=8, max_depth=2, min_samples_split=6, n_jobs=-1),
    "XGBoost": XGBClassifier(
        n_estimators=50, max_depth=3, learning_rate=0.08, colsample_bytree=0.8, subsample=0.9, verbosity=0, use_label_encoder=False
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time

    print(f"{name} model training completed in {train_time:.2f}s.")
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)

    # Save model to "models" directory
    model_path = os.path.join(models_dir, f"{name}_opt.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # Convert to KB

    results[name] = {"Accuracy": acc, "Train Time": train_time, "Model Size (KB)": model_size}
    
    print(f"{name}: Accuracy={acc:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

# Free memory
print("Freeing memory...")
del train_images, train_labels, val_images, val_labels, model
gc.collect()  # Manually trigger garbage collection
print("Memory freed.")

# Output best model
best_model = max(results, key=lambda x: results[x]["Accuracy"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f}")


Loading dataset...
Training Data loaded successfully from NumPy files.
Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Models directory set up.
Training SGD model...
SGD model training completed in 36.70s.
SGD: Accuracy=0.3676, Train Time=36.70s, Model Size=204.21 KB
Training SVM model...


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM model training completed in 332.88s.
SVM: Accuracy=0.3695, Train Time=332.88s, Model Size=325040.61 KB
Training RandomForest model...
RandomForest model training completed in 0.75s.
RandomForest: Accuracy=0.4940, Train Time=0.75s, Model Size=10.44 KB
Training XGBoost model...
XGBoost model training completed in 109.52s.
XGBoost: Accuracy=0.8499, Train Time=109.52s, Model Size=283.03 KB
Freeing memory...
Memory freed.
Best model: XGBoost with Accuracy: 0.8499


In [ ]:
# Preprocessing function (further optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float32') / 255.0  # Normalize
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float32)
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully from NumPy files.")

# Apply optimized preprocessing
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder if it doesn't exist
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)
print("Models directory set up.")

# Define optimized models
models = {
    "SVM": Pipeline([
        ("scaler", MinMaxScaler()), 
        ("svm", LinearSVC(max_iter=3000, tol=1e-4, dual=False))
    ]),
    "RandomForest": RandomForestClassifier(n_estimators=15, max_depth=3, min_samples_split=4, n_jobs=-1),
    "XGBoost": XGBClassifier(
        n_estimators=60, max_depth=3, learning_rate=0.08, 
        colsample_bytree=0.75, subsample=0.9, reg_alpha=0.01, verbosity=0, eval_metric="logloss"
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time

    print(f"{name} model training completed in {train_time:.2f}s.")
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)

    # Save model to "models" directory
    model_path = os.path.join(models_dir, f"{name}_opt1.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # Convert to KB

    results[name] = {"Accuracy": acc, "Train Time": train_time, "Model Size (KB)": model_size}
    
    print(f"{name}: Accuracy={acc:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

# Output best model
best_model = max(results, key=lambda x: results[x]["Accuracy"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f}")


Loading dataset...
Training Data loaded successfully from NumPy files.
Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Models directory set up.
Training SVM model...
SVM model training completed in 558.00s.
SVM: Accuracy=0.3710, Train Time=558.00s, Model Size=305.07 KB
Training RandomForest model...
RandomForest model training completed in 2.02s.
RandomForest: Accuracy=0.5434, Train Time=2.02s, Model Size=29.28 KB
Training XGBoost model...
XGBoost model training completed in 124.57s.
XGBoost: Accuracy=0.8607, Train Time=124.57s, Model Size=337.54 KB
Best model: XGBoost with Accuracy: 0.8607


Comparison table with the actual results from the optimizations:

| Model         | Run | Accuracy | Train Time (s) | Model Size (KB) |
|--------------|-----|----------|---------------|-----------------|
| **SGD**      | 1   | 0.4384   | 27.55         | 224.35          |
| **SVM**      | 1   | 0.4387   | 1291.84       | 326480.53       |
| **RandomForest** | 1   | 0.5397   | 3.99          | 87.19           |
| **XGBoost**  | 1   | 0.7173   | 35.34         | 115.28          |
| **SVM**      | 2   | 0.3812   | 421.04        | 341956.81       |
| **RandomForest** | 2   | 0.5397   | 0.28          | 20.41           |
| **XGBoost**  | 2   | 0.8665   | 72.21         | 370.22          |
| **SVM**      | 3   | 0.3812   | 381.53        | 341956.81       |
| **RandomForest** | 3   | 0.5440   | 0.21          | 20.00           |
| **XGBoost**  | 3   | 0.8940   | 52.03         | 740.09          |
| **SVM**           | 1st Opt. | 0.3667   | 229.88         | 256036.50       |
| **RandomForest**  | 1st Opt. | 0.4946   | 0.21           | 10.44           |
| **XGBoost**       | 1st Opt. | 0.8360   | 15.84          | 282.03          |
| **SVM**           | 2nd Opt. | 0.3670   | 239.53         | 241.32          |
| **RandomForest**  | 2nd Opt. | 0.5440   | 0.26           | 28.47           |
| **XGBoost**       | 2nd Opt. | 0.8471   | 21.57          | 337.08          |

### Best Model

The best model to use is **XGBoost**.

### Reasons:
1. **Highest Accuracy**: XGBoost consistently achieved the highest accuracy across all runs, with the highest accuracy being 0.8940.
2. **Reasonable Training Time**: XGBoost has a reasonable training time compared to SVM, which has significantly longer training times.
3. **Model Size**: The model size of XGBoost is moderate and manageable compared to the extremely large size of the SVM model.

### Considering Accuracy, Time, and Storage:
- **Accuracy**: XGBoost has the highest accuracy (0.8940).
- **Training Time**: XGBoost has a reasonable training time (15.84s to 72.21s).
- **Model Size**: XGBoost has a moderate model size (115.95 KB to 740.09 KB).

Overall, XGBoost provides the best balance of accuracy, training time, and model size, making it the most suitable choice for this task.

### More Tuning to achieve an accurate model to minimize resource use in CubeSat

In [ ]:
# Preprocessing function (optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = X.astype('float16') / 255.0  # Normalize and reduce memory
    if X_pre.ndim == 4 and X_pre.shape[-1] == 3:
        X_pre = np.array([rgb2gray(image) for image in X_pre], dtype=np.float16)  # Convert to grayscale
    X_pre = np.array([resize(image, size, anti_aliasing=False) for image in X_pre], dtype=np.float16)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1)  # Flatten

# Load dataset efficiently
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully.")

# Apply preprocessing
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Define optimized models
models = {
    "SGD": Pipeline([
        ("scaler", MinMaxScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0005, max_iter=500, tol=1e-4, class_weight="balanced", n_jobs=-1))
    ]),
    "SVM": Pipeline([
        ("scaler", StandardScaler()), 
        ("svm", SVC(kernel="linear", probability=False, cache_size=50, max_iter=1000, class_weight="balanced"))
    ]),
    "RandomForest": RandomForestClassifier(
        n_estimators=20, max_depth=4, min_samples_split=5, class_weight="balanced", n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        n_estimators=50, max_depth=3, learning_rate=0.07, colsample_bytree=0.85, subsample=0.95, verbosity=0, use_label_encoder=False
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time

    # Predictions
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)
    f1 = f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    model_path = os.path.join(models_dir, f"{name}4.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # KB

    results[name] = {"Accuracy": acc, "F1-Score": f1, "Train Time": train_time, "Model Size (KB)": model_size}

    print(f"{name}: Accuracy={acc:.4f}, F1-Score={f1:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

    # Free memory after each model
    del model
    gc.collect()

# Select best model based on F1-Score
best_model = max(results, key=lambda x: results[x]["F1-Score"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f} and F1-Score: {results[best_model]['F1-Score']:.4f}")


Loading dataset...
Training Data loaded successfully.
Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Training SGD model...
SGD: Accuracy=0.3979, F1-Score=0.3867, Train Time=23.39s, Model Size=254.85 KB
Training SVM model...


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM: Accuracy=0.3546, F1-Score=0.3542, Train Time=160.34s, Model Size=236544.81 KB
Training RandomForest model...
RandomForest: Accuracy=0.6867, F1-Score=0.6677, Train Time=2.78s, Model Size=62.55 KB
Training XGBoost model...
XGBoost: Accuracy=0.8474, F1-Score=0.8311, Train Time=76.86s, Model Size=282.83 KB
Best model: XGBoost with Accuracy: 0.8474 and F1-Score: 0.8311


In [ ]:
# Preprocessing function (optimized for minimal memory usage)
def preprocessing_fn_ML(X, size=(72, 72)):
    print("Starting preprocessing...")
    X_pre = np.array([resize(rgb2gray(image), size, anti_aliasing=False) for image in X], dtype=np.float32)
    print("Preprocessing completed.")
    return X_pre.reshape(len(X_pre), -1) / 255.0  # Flatten and normalize

# Load dataset efficiently
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print("Training Data loaded successfully.")

# Apply preprocessing
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models folder
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Define optimized models
models = {
    "SGD": Pipeline([
        ("scaler", MinMaxScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0005, max_iter=500, tol=1e-4, class_weight="balanced", n_jobs=-1))
    ]),
    "SVM": Pipeline([
        ("scaler", StandardScaler()), 
        ("svm", SVC(kernel="linear", probability=False, cache_size=50, max_iter=1000, class_weight="balanced"))
    ]),
    "RandomForest": RandomForestClassifier(
        n_estimators=20, max_depth=4, min_samples_split=5, class_weight="balanced", n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        n_estimators=50, max_depth=3, learning_rate=0.07, colsample_bytree=0.85, subsample=0.95, verbosity=0, use_label_encoder=False
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time

    # Predictions
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)
    f1 = f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    model_path = os.path.join(models_dir, f"{name}5.pkl")
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # KB

    results[name] = {"Accuracy": acc, "F1-Score": f1, "Train Time": train_time, "Model Size (KB)": model_size}

    print(f"{name}: Accuracy={acc:.4f}, F1-Score={f1:.4f}, Train Time={train_time:.2f}s, Model Size={model_size:.2f} KB")

    # Free memory after each model
    del model
    gc.collect()

# Select best model based on F1-Score
best_model = max(results, key=lambda x: results[x]["F1-Score"])
print(f"Best model: {best_model} with Accuracy: {results[best_model]['Accuracy']:.4f} and F1-Score: {results[best_model]['F1-Score']:.4f}")


Loading dataset...
Training Data loaded successfully.
Starting preprocessing...
Preprocessing completed.
Starting preprocessing...
Preprocessing completed.
Training SGD model...
SGD: Accuracy=0.3923, F1-Score=0.3839, Train Time=24.24s, Model Size=204.21 KB
Training SVM model...


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM: Accuracy=0.3355, F1-Score=0.3353, Train Time=172.18s, Model Size=235408.95 KB
Training RandomForest model...
RandomForest: Accuracy=0.6793, F1-Score=0.6623, Train Time=2.66s, Model Size=62.75 KB
Training XGBoost model...
XGBoost: Accuracy=0.8480, F1-Score=0.8316, Train Time=102.11s, Model Size=282.96 KB
Best model: XGBoost with Accuracy: 0.8480 and F1-Score: 0.8316


In [ ]:
# Optimized preprocessing function with parallelism
def process_image(image, size=(72, 72)):
    return resize(rgb2gray(image), size, anti_aliasing=False).flatten()

def preprocessing_fn_ML(X):
    return np.array(Parallel(n_jobs=-1)(delayed(process_image)(img) for img in X), dtype=np.float16)

# Load dataset efficiently using mmap_mode
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')

# Apply preprocessing
train_images = preprocessing_fn_ML(train_images)
val_images = preprocessing_fn_ML(val_images)

# Create models directory
os.makedirs("models", exist_ok=True)

# Define optimized models
models = {
    "SGD": Pipeline([
        ("scaler", StandardScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0003, max_iter=400, tol=1e-4, class_weight="balanced", n_jobs=-1))
    ]),
    "SVM": Pipeline([
        ("scaler", StandardScaler()), 
        ("svm", SVC(kernel="linear", probability=False, cache_size=100, max_iter=800, class_weight="balanced"))
    ]),
    "RandomForest": RandomForestClassifier(
        n_estimators=30, max_depth=5, min_samples_split=5, class_weight="balanced", n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        n_estimators=40, max_depth=3, learning_rate=0.08, colsample_bytree=0.9, subsample=0.9, verbosity=0, use_label_encoder=False, n_jobs=-1
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time

    # Predictions
    pred_labels = model.predict(val_images)
    acc = accuracy_score(val_labels, pred_labels)
    f1 = f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    model_path = f"models/{name}6.pkl"
    joblib.dump(model, model_path)

    # Get model size
    model_size = os.path.getsize(model_path) / 1024  # in KB

    results[name] = {"Accuracy": acc, "F1-Score": f1, "Train Time": train_time, "Model Size (KB)": model_size}
    print(f"{name}: Acc={acc:.4f}, F1={f1:.4f}, TrainTime={train_time:.2f}s, ModelSize={model_size:.2f}KB")

    # Free memory
    del model
    gc.collect()

# Select best model based on F1-Score
best_model = max(results, key=lambda x: results[x]["F1-Score"])
print(f"Best model: {best_model} -> Acc={results[best_model]['Accuracy']:.4f}, F1={results[best_model]['F1-Score']:.4f}")


/shared/venv/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD: Acc=0.3855, F1=0.3719, TrainTime=230.49s, ModelSize=325.60KB


/shared/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVM: Acc=0.3556, F1=0.3539, TrainTime=148.70s, ModelSize=211718.17KB
RandomForest: Acc=0.7278, F1=0.6996, TrainTime=5.36s, ModelSize=135.95KB
XGBoost: Acc=0.8418, F1=0.8247, TrainTime=78.32s, ModelSize=227.06KB
Best model: XGBoost -> Acc=0.8418, F1=0.8247


In [ ]:
# Optimized image preprocessing with HOG for better feature extraction
def process_image(image, size=(72, 72)):
    img = resize(rgb2gray(image), size, anti_aliasing=False)
    return hog(img, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)

def preprocessing_fn_ML(X):
    print(f"Processing {len(X)} images...")  # Debugging print
    return np.array(Parallel(n_jobs=2)(delayed(process_image)(img) for img in X), dtype=np.float16)

# Load dataset using mmap_mode to save RAM
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')
print(f"Dataset loaded: {len(train_images)} training images, {len(val_images)} validation images.")

# Apply preprocessing
print("Preprocessing training images...")
train_images = preprocessing_fn_ML(train_images)
print("Preprocessing validation images...")
val_images = preprocessing_fn_ML(val_images)

# Ensure models folder exists
os.makedirs("models", exist_ok=True)

# Define optimized models with lower CPU usage and improved accuracy
models = {
    "SGD": Pipeline([
        ("scaler", StandardScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0002, max_iter=300, tol=1e-4, class_weight="balanced", n_jobs=2))
    ]),
    "RandomForest": RandomForestClassifier(
        n_estimators=20, max_depth=6, min_samples_split=4, class_weight="balanced", n_jobs=2
    ),
    "XGBoost": XGBClassifier(
        n_estimators=30, max_depth=3, learning_rate=0.07, colsample_bytree=0.85, subsample=0.9, verbosity=0, use_label_encoder=False, n_jobs=2
    )
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"\nTraining {name} model...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    print(f"{name} training completed in {train_time:.2f}s.")

    # Predictions and evaluation
    print(f"Evaluating {name} model...")
    pred_labels = model.predict(val_images)
    acc, f1 = accuracy_score(val_labels, pred_labels), f1_score(val_labels, pred_labels, average="weighted")

    # Save model and get size
    model_path = f"models/{name}7.pkl"
    joblib.dump(model, model_path)
    model_size = os.path.getsize(model_path) / 1024  # in KB

    results[name] = {"Acc": acc, "F1": f1, "Train Time": train_time, "Size (KB)": model_size}
    print(f"{name} -> Accuracy: {acc:.4f}, F1-Score: {f1:.4f}, Training Time: {train_time:.2f}s, Model Size: {model_size:.2f}KB")

    # Free memory
    del model
    gc.collect()
    print(f"Memory cleaned up for {name} model.")

# Get best model
best = max(results, key=lambda x: results[x]["F1"])
print(f"\n🏆 Best Model: {best} -> Accuracy: {results[best]['Acc']:.4f}, F1-Score: {results[best]['F1']:.4f}")


Loading dataset...
Dataset loaded: 9711 training images, 3237 validation images.
Preprocessing training images...
Processing 9711 images...
Preprocessing validation images...
Processing 3237 images...

Training SGD model...


/shared/venv/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD training completed in 21.60s.
Evaluating SGD model...
SGD -> Accuracy: 0.8255, F1-Score: 0.8302, Training Time: 21.60s, Model Size: 47.17KB
Memory cleaned up for SGD model.

Training RandomForest model...
RandomForest training completed in 2.11s.
Evaluating RandomForest model...
RandomForest -> Accuracy: 0.7164, F1-Score: 0.6852, Training Time: 2.11s, Model Size: 202.89KB
Memory cleaned up for RandomForest model.

Training XGBoost model...
XGBoost training completed in 5.46s.
Evaluating XGBoost model...
XGBoost -> Accuracy: 0.7461, F1-Score: 0.7103, Training Time: 5.46s, Model Size: 174.91KB
Memory cleaned up for XGBoost model.

🏆 Best Model: SGD -> Accuracy: 0.8255, F1-Score: 0.8302


In [ ]:
# Image Preprocessing with HOG & PCA
def process_image(image, size=(72, 72)):
    img = resize(rgb2gray(image), size, anti_aliasing=False)
    return hog(img, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)

def preprocessing_fn_ML(X, fit_pca=False):
    print(f"Processing {len(X)} images...")
    features = np.array(Parallel(n_jobs=-1)(delayed(process_image)(img) for img in X), dtype=np.float16)
    
    if fit_pca:
        global pca
        print("Fitting PCA...")
        pca = PCA(n_components=100)
        pca.fit(features)
    
    return pca.transform(features)

# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')

# Apply PCA on Training Data, then Transform Validation Data
print("Applying PCA...")
train_images = preprocessing_fn_ML(train_images, fit_pca=True)
val_images = preprocessing_fn_ML(val_images)

# Ensure models folder exists
os.makedirs("models", exist_ok=True)

# Optimized Models
models = {
    "RandomForest": RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=4, class_weight="balanced", n_jobs=-1),
    "XGBoost": XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.05, colsample_bytree=0.9, subsample=0.9, verbosity=0, use_label_encoder=False, n_jobs=-1)
}

# Training & Evaluation
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    print(f"{name} trained in {train_time:.2f}s.")

    # Predictions
    print(f"Evaluating {name}...")
    pred_labels = model.predict(val_images)
    acc, f1 = accuracy_score(val_labels, pred_labels), f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    joblib.dump(model, f"models/{name}8.pkl")

    results[name] = {"Acc": acc, "F1": f1, "Train Time": train_time}
    print(f"{name}: Accuracy={acc:.4f}, F1-Score={f1:.4f}, TrainTime={train_time:.2f}s")

    # Free Memory
    del model
    gc.collect()

# Best Model
best = max(results, key=lambda x: results[x]["F1"])
print(f"\n Best Model: {best} -> Accuracy: {results[best]['Acc']:.4f}, F1-Score: {results[best]['F1']:.4f}")


Loading dataset...
Applying PCA...
Processing 9711 images...
Fitting PCA...
Processing 3237 images...

Training RandomForest...
RandomForest trained in 4.11s.
Evaluating RandomForest...
RandomForest: Accuracy=0.8838, F1-Score=0.8812, TrainTime=4.11s

Training XGBoost...
XGBoost trained in 2.35s.
Evaluating XGBoost...
XGBoost: Accuracy=0.8863, F1-Score=0.8833, TrainTime=2.35s

 Best Model: XGBoost -> Accuracy: 0.8863, F1-Score: 0.8833


In [ ]:
# Image Preprocessing (HOG + PCA)
def process_image(image, size=(72, 72)):
    img = resize(rgb2gray(image), size, anti_aliasing=False)
    return hog(img, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)

def preprocessing_fn_ML(X, fit_pca=False):
    print(f"Processing {len(X)} images...")
    features = np.array(Parallel(n_jobs=-1)(delayed(process_image)(img) for img in X), dtype=np.float16)
    
    if fit_pca:
        global pca
        print("Fitting PCA...")
        pca = PCA(n_components=100)
        pca.fit(features)
    
    return pca.transform(features)

# Load dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')

# Apply PCA on Training Data, then Transform Validation Data
print("Applying PCA...")
train_images = preprocessing_fn_ML(train_images, fit_pca=True)
val_images = preprocessing_fn_ML(val_images)

# Ensure models folder exists
os.makedirs("models", exist_ok=True)

# Optimized Models
models = {
    "SGD": Pipeline([
        ("scaler", StandardScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.0001, max_iter=300, tol=1e-4, class_weight="balanced", n_jobs=-1))
    ]),
    "RandomForest": RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=4, class_weight="balanced", n_jobs=-1),
    "XGBoost": XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.05, colsample_bytree=0.9, subsample=0.9, verbosity=0, use_label_encoder=False, n_jobs=-1)
}

# Training & Evaluation
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    print(f"{name} trained in {train_time:.2f}s.")

    # Predictions
    print(f"Evaluating {name}...")
    pred_labels = model.predict(val_images)
    acc, f1 = accuracy_score(val_labels, pred_labels), f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    joblib.dump(model, f"models/{name}9.pkl")

    results[name] = {"Acc": acc, "F1": f1, "Train Time": train_time}
    print(f"{name}: Accuracy={acc:.4f}, F1-Score={f1:.4f}, TrainTime={train_time:.2f}s")

    # Free Memory
    del model
    gc.collect()

# Best Model
best = max(results, key=lambda x: results[x]["F1"])
print(f"\n Best Model: {best} -> Accuracy: {results[best]['Acc']:.4f}, F1-Score: {results[best]['F1']:.4f}")


Loading dataset...
Applying PCA...
Processing 9711 images...
Fitting PCA...
Processing 3237 images...

Training SGD...
SGD trained in 1.80s.
Evaluating SGD...
SGD: Accuracy=0.8221, F1-Score=0.8167, TrainTime=1.80s

Training RandomForest...
RandomForest trained in 4.10s.
Evaluating RandomForest...
RandomForest: Accuracy=0.8817, F1-Score=0.8791, TrainTime=4.10s

Training XGBoost...
XGBoost trained in 2.33s.
Evaluating XGBoost...
XGBoost: Accuracy=0.8863, F1-Score=0.8833, TrainTime=2.33s

 Best Model: XGBoost -> Accuracy: 0.8863, F1-Score: 0.8833


In [ ]:
# Optimized Image Preprocessing
def process_images(X, fit_pca=False):
    print(f"Processing {len(X)} images...")
    features = np.array([hog(resize(rgb2gray(img), (72, 72), anti_aliasing=False),
                             pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)
                         for img in X], dtype=np.float16)
    
    global pca
    if fit_pca:
        print("Fitting PCA...")
        pca = PCA(n_components=150)
        features = pca.fit_transform(features)
    else:
        features = pca.transform(features)

    return features

# Load Dataset
print("Loading dataset...")
train_images = np.load('data/train_images.npy', mmap_mode='r')
train_labels = np.load('data/train_labels.npy', mmap_mode='r')
val_images = np.load('data/val_images.npy', mmap_mode='r')
val_labels = np.load('data/val_labels.npy', mmap_mode='r')

# Apply Preprocessing & PCA
print("Applying PCA...")
train_images = process_images(train_images, fit_pca=True)
val_images = process_images(val_images)

# Ensure models folder exists
os.makedirs("models", exist_ok=True)

# Optimized Models
models = {
    "SGD": Pipeline([
        ("scaler", StandardScaler()), 
        ("sgd", SGDClassifier(loss="log_loss", alpha=0.00005, max_iter=500, tol=1e-5, class_weight="balanced", n_jobs=1))
    ]),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=50, learning_rate=0.05, max_depth=4),
    "XGBoost": XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.04, colsample_bytree=0.95, subsample=0.95, verbosity=0, use_label_encoder=False, n_jobs=1)
}

# Training & Evaluation
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    model.fit(train_images, train_labels)
    train_time = time.time() - start_time
    print(f"{name} trained in {train_time:.2f}s.")

    # Predictions
    print(f"Evaluating {name}...")
    pred_labels = model.predict(val_images)
    acc, f1 = accuracy_score(val_labels, pred_labels), f1_score(val_labels, pred_labels, average="weighted")

    # Save model
    joblib.dump(model, f"models/{name}10.pkl")

    results[name] = {"Acc": acc, "F1": f1, "Train Time": train_time}
    print(f"{name}: Accuracy={acc:.4f}, F1-Score={f1:.4f}, TrainTime={train_time:.2f}s")

    # Free Memory
    del model
    gc.collect()

# Best Model
best = max(results, key=lambda x: results[x]["F1"])
print(f"\n Best Model: {best} -> Accuracy: {results[best]['Acc']:.4f}, F1-Score: {results[best]['F1']:.4f}")


Loading dataset...
Applying PCA...
Processing 9711 images...
Fitting PCA...
Processing 3237 images...

Training SGD...
SGD trained in 5.54s.
Evaluating SGD...
SGD: Accuracy=0.8282, F1-Score=0.8254, TrainTime=5.54s

Training GradientBoosting...
GradientBoosting trained in 197.06s.
Evaluating GradientBoosting...
GradientBoosting: Accuracy=0.8984, F1-Score=0.8962, TrainTime=197.06s

Training XGBoost...
XGBoost trained in 3.93s.
Evaluating XGBoost...
XGBoost: Accuracy=0.8845, F1-Score=0.8812, TrainTime=3.93s

 Best Model: GradientBoosting -> Accuracy: 0.8984, F1-Score: 0.8962
